<a href="https://www.kaggle.com/code/arihant18/covid-detection?scriptVersionId=181304070" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from tensorflow.keras.utils import image_dataset_from_directory
import keras
from keras.applications import VGG19
from keras.layers import Dense, Flatten
import matplotlib.pyplot as plt

In [ ]:
train='/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train/'
test ='/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/'

In [ ]:
train_ds =image_dataset_from_directory(train,
                                       batch_size=148
                                      )
test_ds =image_dataset_from_directory(test,
                                      batch_size=40
                                      )

In [ ]:
single_batch = train_ds.as_numpy_iterator().next()
train_x, train_y = single_batch[0]/255, single_batch[1]
print(train_x.shape, train_y.shape)

single_batch = test_ds.as_numpy_iterator().next()
test_x, test_y = single_batch[0]/255, single_batch[1]
print(test_x.shape, test_y.shape)

In [ ]:
label=os.listdir(train)
label

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(15,15))
for i in range(4):
    ax[i].axis('off')
    ax[i].imshow(train_x[i])
    ax[i].set_title(label[train_y[i]])

In [ ]:
vgg = VGG19(include_top=False, input_shape=train_x[0].shape)
vgg.summary()

In [ ]:
for layer in vgg.layers:
    layer.trainable = False
print("Number of trainable parameters:",len(vgg.trainable_variables))

In [ ]:
from keras import Model

X = Flatten()(vgg.output)
X = Dense(128, activation='relu')(X)
output = Dense(1, activation='sigmoid')(X)
model = Model(vgg.input, output)
model.summary()

In [ ]:
adam = keras.optimizers.Adam(0.01)
loss = keras.losses.BinaryCrossentropy()

model.compile(loss=loss, optimizer= adam)

In [ ]:
History=model.fit(
            train_x, 
            train_y, 
            batch_size=64, 
            epochs = 30,
            validation_data = (test_x, test_y)         
                 ).history

In [ ]:
plt.title('Loss plot')
plt.plot(History['loss'])
plt.plot(History['val_loss'])
plt.legend(['Train loss', 'Val loss'])
plt.ylabel('Loss')
plt.xlabel('Epochs');

In [ ]:
y_pred = (model.predict(test_x).reshape(-1,)>=0.5).astype(int)
accuracy = np.sum(test_y==y_pred)/len(y_pred)

print("The model have an accuracy of:", accuracy)

In [ ]:
fig,ax= plt.subplots(2,4, figsize=(15, 8))
for i in range(2):
    for j in range(4):
        idx=2*i+j
        ax[i][j].imshow(test_x[idx])
        ax[i][j].axis('off')
        ax[i][j].set_title(f'Actucal: {label[test_y[idx]]}\nPredicted: {label[y_pred[idx]]}')